In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pickle as pkl
from src.utils.data_transform import *

In [2]:
with open('../data/FIC.pkl', 'rb') as fh:
    dataset = pkl.load(fh)

data = dataset['signals_proc']
mm_gt = dataset['mm_gt']
bite_gt = dataset['bite_gt']

In [3]:
# Helper function to create windows from the sensor data
def create_windows(sensor_data, window_size, step_size):
    num_windows = (len(sensor_data) - window_size) // step_size + 1
    return np.array([sensor_data[i:i+window_size] for i in range(0, num_windows * step_size, step_size)])

# Helper function to label the windows based on the micromovements
def label_windows(windows, mm_gt, window_size, step_size):
    labels = np.zeros((len(windows),))
    for start_time, end_time, movement in mm_gt:
        # Find the window index range for the current micromovement
        start_index = int(np.floor(start_time / step_size))
        end_index = int(np.floor(end_time / step_size))
        # Label the windows that overlap with the current micromovement
        labels[start_index:end_index+1] = movement
    return labels

# Parameters for windowing
window_size = 20  # 20 samples per window
step_size = 10    # 10 samples step size
num_classes = 7   # Including the 'other movement' class

# Apply windowing to each meal's sensor data and label the windows
windowed_data = []
windowed_labels = []
for meal_data, meal_mm_gt in zip(data, mm_gt):
    # Extract just the sensor readings, assuming the first column is the timestamp
    sensor_readings = meal_data[:, 1:]
    # Create the windows
    windows = create_windows(sensor_readings, window_size, step_size)
    windowed_data.append(windows)
    # Label the windows
    labels = label_windows(windows, meal_mm_gt, window_size, step_size)
    windowed_labels.append(labels)

# Concatenate all windowed data and labels from all meals
all_windowed_data = np.concatenate(windowed_data, axis=0)
all_windowed_labels = np.concatenate(windowed_labels, axis=0)

# One-hot encode the labels
encoded_labels = to_categorical(all_windowed_labels, num_classes=num_classes)


In [ ]:
# # Parameters
# window_length = 20  # Number of samples in each window (0.2 seconds at 100Hz)
# step_size = 10      # Step size for the sliding window (0.1 seconds at 100Hz)
# 
# # Initialize lists to store windowed data and labels
# windowed_data = []
# windowed_labels = []

In [ ]:
# def label_window(start, end, meal_mm_gt):
#     # Count the occurrences of each micromovement type within the window
#     counts = {}
#     for mm in meal_mm_gt:
#         mm_start, mm_end, mm_type = mm
#         if mm_start >= end:
#             break
#         if mm_end <= start:
#             continue
#         counts[mm_type] = counts.get(mm_type, 0) + 1
# 
#     # Return the most frequent micromovement type, or 0 if the window is empty
#     return max(counts, key=counts.get) if counts else 0


In [ ]:
# for meal_data, meal_mm_gt in zip(data, mm_gt):
#     # Remove the timestamp column (assuming it's the first column)
#     meal_data_no_timestamp = meal_data[:, 1:]
# 
#     # Apply the sliding window
#     for start in range(0, len(meal_data_no_timestamp) - window_length + 1, step_size):
#         end = start + window_length
#         window = meal_data_no_timestamp[start:end]
# 
#         # Label the window
#         label = label_window(start, end, meal_mm_gt)
#         windowed_data.append(window)
#         windowed_labels.append(label)

In [ ]:
# windowed_data = np.array(windowed_data)
# windowed_labels = np.array(windowed_labels)

In [ ]:
# len(windowed_data)

In [ ]:
# windowed_labels

In [ ]:
# def label_windows(windowed_data, mm_gt, window_length_samples=20, sampling_rate=100):
#     labels = []
#     # Convert timestamps to sample indices
#     mm_gt_indices = (mm_gt[:, :2] * sampling_rate).astype(int)
#     for window in windowed_data:
#         # Use the middle point of the window to determine the label
#         middle_sample_index = window[0, 0] * sampling_rate + window_length_samples // 2
#         label = 0  # Default to 'no movement'
#         for start_idx, end_idx, movement in mm_gt_indices:
#             if start_idx <= middle_sample_index <= end_idx:
#                 label = movement
#                 break
#         # Exclude 'other movement' category
#         if label != 6:
#             labels.append(label)
#         else:
#             labels.append(0)  # Treat 'other movement' as 'no movement'
#     return np.array(labels) - 1  # Subtract 1 to match the output layer indexing from 0 to 4


In [ ]:
# def create_windows(subject_data, window_length=20, step_size=10):
#     windows = []
#     for i in range(0, subject_data.shape[0] - window_length + 1, step_size):
#         window = subject_data[i:i+window_length, 1:]  # Exclude the timestamp column
#         windows.append(window)
#     return np.array(windows)

In [ ]:
# all_windows = []
# all_labels = []

In [ ]:
# for subject_data, subject_mm_gt in zip(data, mm_gt):
#     # Create windows for the subject's data
#     subject_windows = create_windows(np.array(subject_data))
#     # Label the windows for the subject
#     subject_labels = label_windows(subject_windows, np.array(subject_mm_gt))
#     all_windows.extend(subject_windows)
#     all_labels.extend(subject_labels)

In [ ]:
# all_windows = np.array(all_windows)
# all_labels = np.array(all_labels)

In [ ]:
# all_labels_one_hot = to_categorical(all_labels, num_classes=5)

In [4]:
# CNN model architecture
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=10, activation='relu', input_shape=(20, 6), padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=10, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.5))  # Apply dropout before the fully connected layer
model.add(Dense(5, activation='softmax'))  # 5 classes for the output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
# model.summary()

2024-01-03 18:47:12.860476: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-01-03 18:47:12.860513: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-01-03 18:47:12.860521: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-01-03 18:47:12.860569: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-03 18:47:12.860590: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# Train the CNN with the windowed data and labels
model.fit(all_windowed_data, all_windowed_labels, epochs=32, batch_size=32)

ValueError: Data cardinality is ambiguous:
  x sizes: 163106
  y sizes: 6
Make sure all arrays contain the same number of samples.

In [8]:
all_windowed_labels

array([0., 6., 4., ..., 0., 0., 0.])

In [14]:
new_windowed = all_windowed_data[all_windowed_labels != 6]
new_labels = all_windowed_labels[all_windowed_labels != 6]

162195

In [17]:
model.fit(new_windowed, new_labels, epochs=32, batch_size=32)


Epoch 1/32


ValueError: in user code:

    File "/opt/homebrew/Caskroom/miniforge/base/envs/tfenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/tfenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/homebrew/Caskroom/miniforge/base/envs/tfenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/tfenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/tfenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return 